# **Tugas 2 : Implementasi hasil VSM dengan algoritma Logistic Regression**

Dibuat Oleh:

* Nama : Zainal Arifin
* NIM : 220411100103
* Kelas : PPW A

## **Persiapan Library**

In [ ]:
!pip install Sastrawi
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re

from tqdm import tqdm

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
# library untuk proses modeling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# library untuk evaluasi model
from sklearn.metrics import classification_report, confusion_matrix

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# save model
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


* Preprocessing: Mengubah label kategori menjadi numerik agar bisa diproses oleh model.

* train_test_split: Membagi dataset menjadi data training dan testing.

* LogisticRegression: Digunakan untuk membuat model prediksi dengan algoritma Logistic Regression.

* classification_report dan confusion_matrix: Evaluasi performa model melalui metrik seperti precision, recall, F1-score, dan matriks kebingungan.

* Matplotlib dan Seaborn: Plotting dan visualisasi data.

* Pickle: Menyimpan model yang sudah dilatih agar bisa digunakan tanpa training ulang.

## **Load Dataset**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PPW A/crawl_berita.csv', delimiter=',')
df

,Judul,Isi,Tanggal,Kategori
0,Pertemuan Sri Mulyani-Prabowo Tak Banyak Bahas...,Wakil Menteri Keuangan II Thomas Djiwandono me...,"Rabu, 11 Sep 2024 18:10 WIB",Ekonomi
1,Pebisnis Minta Jokowi Cabut Larangan Jual Roko...,Gabungan pengusaha rokok dan petani tembakau m...,"Rabu, 11 Sep 2024 17:31 WIB",Ekonomi
2,IHSG Melemah Tipis ke 7.760 Sore Ini,Indeks Harga Saham Gabungan (IHSG) ditutup di ...,"Rabu, 11 Sep 2024 16:37 WIB",Ekonomi
3,Rupiah Menguat Rp15.402 per Dolar AS Usai Deba...,Nilai tukar rupiah berada di level Rp15.402 pe...,"Rabu, 11 Sep 2024 16:24 WIB",Ekonomi
4,Sri Mulyani Usai Nonton Timnas-Australia: Teri...,Menteri Keuangan Sri MulyaniÂ berkomentar soal...,"Rabu, 11 Sep 2024 15:47 WIB",Ekonomi
...,...,...,...,...
95,Hasil Liga 1: PSM vs Persib Sama Kuat,PSM Makassar harus puas berbagi satu angka usa...,"Rabu, 11 Sep 2024 17:25 WIB",Olahraga
96,"Jokowi Beri Bonus Rp36,25 Miliar ke Peraih Med...",Presiden Joko Widodo (Jokowi) menyerahkan bonu...,"Rabu, 11 Sep 2024 17:13 WIB",Olahraga
97,Megawati Ungkap Target di Liga Korea: Jadi Pem...,Megawati Hangestri Pertiwi mengungkapkan targe...,"Rabu, 11 Sep 2024 16:49 WIB",Olahraga
98,Media Vietnam: Indonesia Buat Kejutan Besar La...,Media Vietnam memuji performa Timnas Indonesia...,"Rabu, 11 Sep 2024 16:24 WIB",Olahraga


In [ ]:
def web_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
def get_element_text(driver, xpath):
    try:
        return WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text.strip()
    except Exception as e:
        print(f"Error finding element with XPath {xpath}: {e}")
        return ""

In [ ]:
def extract_article_content(driver, article_url):
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1'))
    )
    title = get_element_text(driver, './/h1')

    date = get_element_text(driver, './/p[@class="pt-20 date"]')

    content_elements = driver.find_elements(By.XPATH, './/div[@class="news-text"]/p')
    content = " ".join(p.text for p in content_elements)

    return {
        "Title": title,
        "Date": date,
        "Content": content
    }

In [ ]:
def extract_category_data(driver):
    articles = driver.find_elements(By.XPATH, '//div[@class="item"]')
    article_urls = [article.find_element(By.XPATH, './/a').get_attribute('href') for article in articles]
    return article_urls

def scrape_category_page(driver, main_url, category_name, max_articles=1):
    all_article_data = []
    page_num = 1

    while len(all_article_data) < max_articles:
        current_url = f"{main_url}?page={page_num}"
        driver.get(current_url)

        article_urls = extract_category_data(driver)

        if not article_urls:
            print(f"No more articles found on page {page_num}. Stopping.")
            break

        for url in article_urls:
            if len(all_article_data) >= max_articles:
                break

            article_data = extract_article_content(driver, url)
            article_data["Category"] = category_name
            all_article_data.append(article_data)
        page_num += 1
        time.sleep(2)

    return all_article_data

driver = web_driver()

kategori_list = [
    # {"url": "https://www.metrotvnews.com/channel/ekonomi", "name": "Ekonomi"},
    {"url": "https://www.metrotvnews.com/channel/olahraga", "name": "Olahraga"}
]

all_data = []

for category in kategori_list:
    print(f"Scraping category: {category['name']}")
    category_data = scrape_category_page(driver, category["url"], category["name"], max_articles=1)
    all_data.extend(category_data)

driver.quit()

Scraping category: Olahraga


In [ ]:
df = pd.DataFrame(all_data)
df

,Title,Date,Content,Category
0,Pelatih Satria Muda Geram Timnya Lambat Panas,Gregorius Gelino • 5 October 2024 00:13,Jakarta: Satria Muda Pertamina Jakarta memenan...,Olahraga


In [ ]:
df.to_csv('hasil_crawling.csv', index=False)

In [ ]:
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

df['lower case'] = df['Content'].apply(clean_lower)

casefolding = pd.DataFrame(df['lower case'])
casefolding

,lower case
0,jakarta: perusahaan logistik dhl express mempe...
1,jakarta: bank indonesia (bi) bakal meluncurkan...
2,menteri perhubungan budi karya sumadi mengungk...
3,jakarta: menteri perdagangan (mendag) zulkifli...
4,jakarta: ekonom dari bright institute awalil r...
5,solo: ponsianus nyoman indrawan menjadikan kes...
6,jakarta: ketua umum pssi erick thohir tidak ad...
7,jakarta: dpr telah menyetuji naturalisasi elia...
8,bandung: david liberty nuban atau yang akrab d...
9,bandung: tangerang hawks terpaksa harus merela...


In [ ]:
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z ]')
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df['tanda baca'] = df['lower case'].apply(clean_punct)

df['tanda baca']

,tanda baca
0,jakarta perusahaan logistik dhl express memper...
1,jakarta bank indonesia bi bakal meluncurkan in...
2,menteri perhubungan budi karya sumadi mengungk...
3,jakarta menteri perdagangan mendag zulkifli ha...
4,jakarta ekonom dari bright institute awalil ri...
5,solo ponsianus nyoman indrawan menjadikan kesa...
6,jakarta ketua umum pssi erick thohir tidak ada...
7,jakarta dpr telah menyetuji naturalisasi elian...
8,bandung david liberty nuban atau yang akrab di...
9,bandung tangerang hawks terpaksa harus merelak...


In [ ]:
def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

df['spasi'] = df['tanda baca'].apply(_normalize_whitespace)

df['spasi']

,spasi
0,jakarta perusahaan logistik dhl express memper...
1,jakarta bank indonesia bi bakal meluncurkan in...
2,menteri perhubungan budi karya sumadi mengungk...
3,jakarta menteri perdagangan mendag zulkifli ha...
4,jakarta ekonom dari bright institute awalil ri...
5,solo ponsianus nyoman indrawan menjadikan kesa...
6,jakarta ketua umum pssi erick thohir tidak ada...
7,jakarta dpr telah menyetuji naturalisasi elian...
8,bandung david liberty nuban atau yang akrab di...
9,bandung tangerang hawks terpaksa harus merelak...


In [ ]:
def clean_stopwords(text):
    if isinstance(text, str):
        stopword = set(stopwords.words('indonesian'))
        text = ' '.join(word for word in text.split() if word not in stopword)
        return text.strip()
    return text

df['stopwords'] = df['spasi'].apply(clean_stopwords)

df['stopwords']

,stopwords
0,jakarta perusahaan logistik dhl express memper...
1,jakarta bank indonesia bi meluncurkan infrastr...
2,menteri perhubungan budi karya sumadi tahap up...
3,jakarta menteri perdagangan mendag zulkifli ha...
4,jakarta ekonom bright institute awalil rizky k...
5,solo ponsianus nyoman indrawan menjadikan kesa...
6,jakarta ketua pssi erick thohir salah upaya na...
7,jakarta dpr menyetuji naturalisasi eliano reij...
8,bandung david liberty nuban akrab disapa papen...
9,bandung tangerang hawks terpaksa merelakan ted...


In [ ]:
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

df['stemming'] = df['stopwords'].apply(sastrawistemmer)

df['stemming']

100%|██████████| 110/110 [00:03<00:00, 36.44it/s]


,stemming
0,jakarta usaha logistik dhl express kenal rute ...
1,jakarta bank indonesia bi luncur infrastruktur...
2,menteri hubung budi karya sumadi tahap upaya h...
3,jakarta menteri dagang mendag zulkifli hasan h...
4,jakarta ekonom bright institute awalil rizky p...
5,solo ponsianus nyoman indrawan jadi kesatria b...
6,jakarta ketua pssi erick thohir salah upaya na...
7,jakarta dpr tuji naturalisasi eliano reijnders...
8,bandung david liberty nuban akrab sapa papen r...
9,bandung tangerang hawks paksa rela teddy apriy...


In [ ]:
count_vectorizer = CountVectorizer()

if 'stemming' in df.columns:
    corpus = df['stemming'].tolist()
else:
    raise KeyError("Kolom 'stemming' tidak ada di DataFrame")

x_count = count_vectorizer.fit_transform(corpus)
feature_names = count_vectorizer.get_feature_names_out()
x_count_df = pd.DataFrame(x_count.toarray(), columns=feature_names)

x_count_df

,abraham,advisor,ahmad,air,akrab,akun,alam,alami,ambil,andal,...,vietnam,volatilitas,volume,wacana,webinar,wenas,widodo,wni,zulhas,zulkifli
0,0,1,4,0,0,0,1,0,0,1,...,1,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,1,0,6,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,3,0,...,0,0,0,0,0,0,0,3,0,0
8,0,0,1,0,1,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Load the saved model from file
filename = '/content/drive/MyDrive/PPW A/tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

In [ ]:
corpus = df['stemming'].tolist()
x_tfidf = tfidf_vectorizer.transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=feature_names)
cat_df = df["Category"]
tfidf_df['Category'] = cat_df.values
tfidf_df = tfidf_df[['Category'] + [col for col in tfidf_df.columns if col != 'Category']]

tfidf_df

,Category,abadi,abbeele,abdullah,abilitysport,abilitysprot,abinisa,abraham,absen,acara,...,youth,yunani,yusrizal,za,zakiy,zaman,zeki,zian,zona,zuri
0,Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.097,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
tfidf_df['Category'] = label_encoder.fit_transform(tfidf_df['Category'])

tfidf_df

,Category,abadi,abbeele,abdullah,abilitysport,abilitysprot,abinisa,abraham,absen,acara,...,youth,yunani,yusrizal,za,zakiy,zaman,zeki,zian,zona,zuri
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.097,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Load the saved model from file
filename = '/content/drive/MyDrive/PPW A/lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_test = tfidf_df['Category']
x_test = tfidf_df.drop(['Category'], axis=1)
y_pred = lr_model.predict(x_test)

print(y_pred)

[0 0 0 0 0 1 1 1 1 1]


In [ ]:
# melihat nilai actual dan predicted
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a

,Actual value,Predicted value
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1
